# Scraper for the following website:
##### https://www.survivingantidepressants.org/

The scraper requires a root directory which has all the links containing discussion threads for a particular area. An example is https://www.survivingantidepressants.org/forum/8-symptoms-and-self-care/ 

Then, the scraper is created to record every single URL and iterate through it. The scraper scrapes the content and the date of each comment in discussion threads and adds them to a list. Eventually, this is transformed to a dataframe and added to the main dataframe. 

### Languages and frameworks used:
* Python
* Selenium using Python for scraping and web automation
* Pandas for collating data

### The scraper has the following functions:
* generate_url(driver) -> list: This function takes in a root URL and scrapes every single URL in it from the first page to the last page. These URLs are used by the scrape_a_site() function to scrape. It returns a list containing URLs as strings.
    * Parameters:
        * driver: Selenium WebDriver element
* scrape_a_page(all_dates_list, all_comments_list, driver) -> pandas DataFrame Object: This function scrapes a page. 
    * Parameters:
        * all_dates_list: A list containing dates for any past pages in a particular discussion. 
        * all_comments_list: A list containing the comments for any past pages for a particular discussion
        * driver: Selenium WebDriver element
* scrape_a_site(url, driver): The function uses the scrape_a_page method to scrape a whole discussion thread. Since a site is made up of various pages, the previous method plays a big part. At the end, it collates all comments, and dates scraped to create a dataframe which gets returned. This dataframe contains all records of a particular discussion post/thread.
    * Parameters:
        * url: A string which gets passed to load the Selenium WebDriver
        * driver: Selenium WebDriver element

# Imports

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import time

# scrape_a_page method:

adds all comments and dates scrapes to the parameters all_dates_list and all_comments_list

In [49]:
def scrape_a_page(all_dates_list, all_comments_list, driver):
    comments_list = driver.find_elements(By.XPATH, '//div[starts-with(@id, "comment")]')
    
    if (len(driver.find_elements(By.XPATH, '//a[@data-page="1"]')) > 0): 
        if driver.find_element(By.XPATH, '//a[@data-page="1"]'):
            del comments_list[0]
    
    for comment in comments_list:
        #path_to_child_element = f'//div[starts-with(@id, "comment")]//child::p'
        text_list = comment.find_elements(By.XPATH, './/div[@data-role="commentContent"]')
        date = comment.find_element(By.XPATH, './/time')
        text_in_str = ''
        for text in text_list:
            text_in_str += text.text
        all_dates_list.append(date.text)
        all_comments_list.append(text_in_str)
    


# scrape_a_site 

Scrapes the entire discussion post/thread with the scrape_a_page method. At the end, creates a pandas dataframe to return so it can be added to the main dataframe

In [50]:
def scrape_a_site(url, driver):
    
    driver.get(url)
    
    all_dates_list = []
    all_comments_list = []

    while True:
        scrape_a_page(all_dates_list, all_comments_list, driver)
        if len(driver.find_elements(By.XPATH, 
            '//li[@class="ipsPagination_next ipsPagination_inactive"]')) > 0:
            break
        if len(driver.find_elements(By.XPATH, '//a[@rel="next"]')) > 0:
            next_page = driver.find_element(By.XPATH, '//a[@rel="next"]')
            driver.get(next_page.get_attribute('href'))
        else:
            break
            
    #make tuples
    data_tuples = list(zip(all_dates_list, all_comments_list))
    df_to_return = pd.DataFrame(data_tuples, columns=['Date', 'Comment'])
    return df_to_return

    

# generate_url
Scrapes the root directory containing all discussions and posts. Returns a list of URLs to scrape.

In [51]:
def generate_url(driver):
    list_of_urls = []
    while True:
        list_of_urls_on_page = driver.find_elements(By.XPATH, '//span[@class="ipsType_break ipsContained"]/a')
        for ele in list_of_urls_on_page:
            url = ele.get_attribute('href')
            list_of_urls.append(url)
        if len(driver.find_elements(By.XPATH, '//li[@class="ipsPagination_next ipsPagination_inactive"]')) > 0:
            return list_of_urls
        next_page = driver.find_element(By.XPATH, '//a[@rel="next"]')
        driver.get(next_page.get_attribute('href'))
        


# main function
Driver method. Instantiates the browser and driver so that Selenium can operate. Calls generate_url() and saves the result as a list. Then, creates the main DataFrame and iterates over the list to scrape each site using the above methods. As each site is scraped, the resulting DataFrame returned is appended onto the main DataFrame

In [52]:
browser=path = r'C:\Users\athar\Downloads\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(path)

root_url = 'https://www.survivingantidepressants.org/forum/8-symptoms-and-self-care/'
driver.get(root_url)
url_list = generate_url(driver)
print(len(url_list))

#make dataframe
df = pd.DataFrame(columns=['Date', 'Comment'])


for url in url_list:
    df_to_add = scrape_a_site(url, driver)
    #add to existing dataframe
    df = pd.concat([df, df_to_add], axis=0, ignore_index=True)
    print(df)


C:\Users\athar\AppData\Local\Temp\ipykernel_14252\239384767.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


1544
                  Date                                            Comment
0      October 9, 2012  ADMIN NOTE This topic is a general discussion ...
1        March 8, 2013  So I just read the post on stabilizing...\n  "...
2        March 8, 2013  Sorry for the confusion, cmusic's question was...
3       April 10, 2013  My little experience: Jumped off Zyprexa 12/29...
4       April 16, 2013  Some people get over withdrawal symptoms such ...
..                 ...                                                ...
161  December 23, 2021    On 12/23/2021 at 11:36 AM, ChessieCat said:\...
162  December 23, 2021    On 12/23/2021 at 11:41 AM, curiousnootropics...
163  December 23, 2021    On 12/23/2021 at 11:53 AM, ChessieCat said:\...
164  December 24, 2021    On 12/23/2021 at 10:01 PM, rilexapro said:\n...
165           March 17    On 4/9/2019 at 12:58 PM, Lloyd said:\n    He...

[166 rows x 2 columns]
                Date                                            Comment
0    Octobe

                 Date                                            Comment
0     October 9, 2012  ADMIN NOTE This topic is a general discussion ...
1       March 8, 2013  So I just read the post on stabilizing...\n  "...
2       March 8, 2013  Sorry for the confusion, cmusic's question was...
3      April 10, 2013  My little experience: Jumped off Zyprexa 12/29...
4      April 16, 2013  Some people get over withdrawal symptoms such ...
...               ...                                                ...
2499  August 10, 2022    On 8/10/2022 at 4:34 AM, Greatful said:\nI c...
2500  August 10, 2022    On 8/10/2022 at 4:34 AM, Greatful said:\n@Da...
2501  August 10, 2022  @DazzlingCold @Greatful\n  What both of you ha...
2502  August 10, 2022    On 8/10/2022 at 4:55 AM, Ariel said:\nIs it ...
2503  August 10, 2022  @DazzlingCold@Greatful @Ariel\nwhat y’all said...

[2504 rows x 2 columns]
                  Date                                            Comment
0      October 9, 2012  A

                     Date                                            Comment
0         October 9, 2012  ADMIN NOTE This topic is a general discussion ...
1           March 8, 2013  So I just read the post on stabilizing...\n  "...
2           March 8, 2013  Sorry for the confusion, cmusic's question was...
3          April 10, 2013  My little experience: Jumped off Zyprexa 12/29...
4          April 16, 2013  Some people get over withdrawal symptoms such ...
...                   ...                                                ...
4418     October 22, 2022    On 8/31/2012 at 2:51 AM, Shanti said:\nMotio...
4419  Tuesday at 07:47 AM    On 1/5/2013 at 5:52 AM, starlitegirlx said:\...
4420                       Hello all,\nI am wondering how I might adjust ...
4421             March 20  Hello all,\nI am wondering how I might adjust ...
4422  Tuesday at 04:54 AM  Hello Poostyx.  I'm into the same situation as...

[4423 rows x 2 columns]
                    Date                           

                 Date                                            Comment
0     October 9, 2012  ADMIN NOTE This topic is a general discussion ...
1       March 8, 2013  So I just read the post on stabilizing...\n  "...
2       March 8, 2013  Sorry for the confusion, cmusic's question was...
3      April 10, 2013  My little experience: Jumped off Zyprexa 12/29...
4      April 16, 2013  Some people get over withdrawal symptoms such ...
...               ...                                                ...
5775         March 16    On 3/14/2023 at 7:27 PM, Estman said:\nUsual...
5776         March 16    On 3/16/2023 at 7:23 AM, tsranga said:\nThis...
5777         March 17  The body is way too complex to get answers to ...
5778         March 18    On 3/17/2023 at 10:44 AM, tsranga said:\nThe...
5779         March 18  If you keep up with this level of analysis, th...

[5780 rows x 2 columns]
                  Date                                            Comment
0      October 9, 2012  A

...

                  Date                                            Comment
0      October 9, 2012  ADMIN NOTE This topic is a general discussion ...
1        March 8, 2013  So I just read the post on stabilizing...\n  "...
2        March 8, 2013  Sorry for the confusion, cmusic's question was...
3       April 10, 2013  My little experience: Jumped off Zyprexa 12/29...
4       April 16, 2013  Some people get over withdrawal symptoms such ...
...                ...                                                ...
42557  August 19, 2011  I was thinking about 'recovery stories' today....
42558  August 20, 2011  That's a great idea, Ajay!! I could really use...
42559                   Is it helpful, I feel that my body is very wee...
42560  August 14, 2011  Is it helpful, I feel that my body is very wee...
42561  August 15, 2011  Soso, you will have to use Google and decide f...

[42562 rows x 2 columns]
                  Date                                            Comment
0      Octob

                  Date                                            Comment
0      October 9, 2012  ADMIN NOTE This topic is a general discussion ...
1        March 8, 2013  So I just read the post on stabilizing...\n  "...
2        March 8, 2013  Sorry for the confusion, cmusic's question was...
3       April 10, 2013  My little experience: Jumped off Zyprexa 12/29...
4       April 16, 2013  Some people get over withdrawal symptoms such ...
...                ...                                                ...
42638     June 6, 2011    On 6/6/2011 at 5:14 AM, 'summer' said:\nHey ...
42639     June 6, 2011    On 6/6/2011 at 5:22 AM, 'Hopeful' said:\n  O...
42640     June 7, 2011  Hey, Buxy! Though it's not directly connected ...
42641    July 15, 2011  I have had the same problems in the last year ...
42642    July 15, 2011  Hello, adawn. Welcome to Surviving Antidepress...

[42643 rows x 2 columns]
                  Date                                            Comment
0      Octob

                  Date                                            Comment
0      October 9, 2012  ADMIN NOTE This topic is a general discussion ...
1        March 8, 2013  So I just read the post on stabilizing...\n  "...
2        March 8, 2013  Sorry for the confusion, cmusic's question was...
3       April 10, 2013  My little experience: Jumped off Zyprexa 12/29...
4       April 16, 2013  Some people get over withdrawal symptoms such ...
...                ...                                                ...
42713    June 28, 2011  MAny people talk about asymmetric issues annoy...
42714     July 3, 2011    On 6/28/2011 at 6:54 PM, 'Claudius' said:\nM...
42715     July 3, 2011  http://www.arachnoiditis.info/website_captures...
42716     July 3, 2011  hi Claudius,\n  you said recently you have pai...
42717     July 3, 2011    My left eye keeps twitching too. Sooo annoying!

[42718 rows x 2 columns]
                  Date                                            Comment
0      Octob

                  Date                                            Comment
0      October 9, 2012  ADMIN NOTE This topic is a general discussion ...
1        March 8, 2013  So I just read the post on stabilizing...\n  "...
2        March 8, 2013  Sorry for the confusion, cmusic's question was...
3       April 10, 2013  My little experience: Jumped off Zyprexa 12/29...
4       April 16, 2013  Some people get over withdrawal symptoms such ...
...                ...                                                ...
42786    June 13, 2011  Hi Stan,\nyeah this is one of the exercises th...
42787    June 16, 2011  I got up one night to go to the bathroom - fel...
42788    June 16, 2011  Over and above many vestibular imbalances duri...
42789    June 17, 2011  hi Squirrel,\n  when i walked my 5 daily stree...
42790    June 17, 2011                Ah Stan Thank you that is so sweet.

[42791 rows x 2 columns]
                  Date                                            Comment
0      Octob

                  Date                                            Comment
0      October 9, 2012  ADMIN NOTE This topic is a general discussion ...
1        March 8, 2013  So I just read the post on stabilizing...\n  "...
2        March 8, 2013  Sorry for the confusion, cmusic's question was...
3       April 10, 2013  My little experience: Jumped off Zyprexa 12/29...
4       April 16, 2013  Some people get over withdrawal symptoms such ...
...                ...                                                ...
42890   April 10, 2011    On 4/9/2011 at 11:38 PM, 'alexejice' said:\n...
42891                   I got the inspiration for this thread from thi...
42892   March 28, 2011  I got the inspiration for this thread from thi...
42893   March 29, 2011  Good question, cinephile. And that's a good pr...
42894   March 30, 2011  If you're taking a bunch of supplements, put a...

[42895 rows x 2 columns]
                  Date                                            Comment
0      Octob

In [54]:
len(df)

42907

# Convert Generated DataFrame to csv file

In [60]:
df.to_csv('symptoms_and_withdrawals.csv', index=False)

In [61]:
check_df = pd.read_csv('symptoms_and_withdrawals.csv')

In [62]:
check_df

,Date,Comment
0,"October 9, 2012",ADMIN NOTE This topic is a general discussion ...
1,"March 8, 2013","So I just read the post on stabilizing...\n ""..."
2,"March 8, 2013","Sorry for the confusion, cmusic's question was..."
3,"April 10, 2013",My little experience: Jumped off Zyprexa 12/29...
4,"April 16, 2013",Some people get over withdrawal symptoms such ...
...,...,...
42902,NaN,I wish this was true for me. What's your exper...
42903,"March 19, 2011",I wish this was true for me. What's your exper...
42904,"March 20, 2011","It's been ages since I had the ""Glued-Eyes"" Sy..."
42905,"March 20, 2011","Neuro, was there a period during your withdraw..."
